In [ ]:
# Model design
import agentpy as ap
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
import math

# List of moves in roundabout
POS = [1,2,3,4]
MOVES = [(0, 1), (0, -1), (1, 0), (-1,0), (-1, 1), (-1, -1)]

In [ ]:
class RoundAboutModel(ap.Model):
    
    def drawCircle(self, x, y, radius):
        do = []
        Epsilon = 2.2
        for ejeY in range(x-radius,x+radius+1):
            for ejeX in range(y-radius,y+radius+1):
                # see if we're close to (x-a)**2 + (y-b)**2 == r**2
                if abs((ejeX-x)**2 + (y-ejeY)**2 - radius**2) < Epsilon**2:
                    do.append((ejeX,ejeY))
        return do
        
    def resetNeighbors(self,carro):
        if len(self.carTable[carro.id]) != 0:
            self.carTable[carro.id][0].condition = 0
            self.carTable[carro.id].pop(0)
            
    def setup(self):
        self.carTable = {}

        n_cars= int(self.p['Densidad'] * (self.p.M**2))
        cars = self.agents = ap.AgentList(self, n_cars)
        self.carros = ap.AgentList(self, self.p['nCars'])
        # Defining grid size
        self.street = ap.Grid(self, [self.p.M] * 2, track_empty=True)

        # Add agents to grid 
        self.street.add_agents(self.carros,[(2,21)]*self.p['nCars'])

        pos_count = self.p.M -4
        self.carros.position = 0
        self.carros.inside = 0
        self.carros.id = 0
        id = 0
        for carro in self.carros:
            carro.id = id
            id += 1
            self.carTable[carro.id] = []
            carro.position = random.choice(POS)
            self.street.move_to(carro, (37, pos_count))
            pos_count -= 5
        
        self.agents.condition = 0
        self.getStreetPositions()
        
    def getStreetPositions(self):
        routePoints = self.drawCircle(int(self.p.M/2),int(self.p.M/2), 7)
        self.road = routePoints
        self.roundabout = ap.AgentList(self,len(routePoints)*2)

        self.street.add_agents(self.roundabout,routePoints)
        
        self.roundabout.condition = 0
        self.roundabout.exit = -1
        
        #exit setup
        self.roundabout[1].exit = 0
        self.roundabout[21].exit = 3
        self.roundabout[18].exit = 1
        self.roundabout[38].exit = 2
        # draw cars
        self.carros.condition = 1
        self.carros.destination = -1
        #Agent_type: {0:space, 1:street, 2: car}

    #def getStreetPositions(self):
        #positions= []
        #for index in range(len(self.car)):
        #    if index % 2 == 0:
        #        positions.append((30,0+index))
        #    else:
        #        positions.append((0+index,30))
        #return positions


    #def step(self):
     #   carros = self.carro
      #  for i, carro in enumerate(carros):
       #     if i % 2  == 0:
        #        self.street.move_by(carro, (0,1))
         #   else:
          #      self.street.move_by(carro,(1,0))



    def end(self):
        # Document a measure at the end of the simulation
        self.report('Percentage of cleaned tiles',2)
        print(self.report['Percentage of cleaned tiles'])

In [ ]:
# Define parameters

parameters = {
    'Densidad': 1, # Percentage of grid covered by trees
    'M': 50, # city size
    'steps': 50, # Number of simulation steps 
    'nCars': 10, # Number of cleaning robot agents
}

In [ ]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#808080', 1:'#d62c2c', 2:'#808080', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Roundabout Model Simulation\n"
                 f"Time-step: {model.t}, Cars: "
                 f"{len(model.carros)}")
    

fig, ax = plt.subplots()

circulo = plt.Circle((20,20), 10, color='b')

ax.add_patch(circulo)
model = RoundAboutModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=50))